In [2]:
import requests
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

In [ ]:
endpoint = 'https://1507-computervision-we.cognitiveservices.azure.com/'
api_key = '450e09da0b934eb2bc950eb9f51fa754'

client = ImageAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(api_key)
)

result = client.analyze_from_url(
    image_url="https://learn.microsoft.com/azure/ai-services/computer-vision/media/quickstarts/presentation.png",
    visual_features=[VisualFeatures.CAPTION, VisualFeatures.READ],
    gender_neutral_caption=True,  # Optional (default is False)
)

print(result)

In [3]:
import gradio as gr
from PIL import Image, ImageDraw, ImageFont

def request_vision(image_path):
    features = 'objects,tags,SmartCrops'
    endpoint = f'https://1507-computervision-we.cognitiveservices.azure.com/computervision/imageanalysis:analyze?api-version=2024-02-01&model-name=kitchendetection&features={features}'
    api_key = '450e09da0b934eb2bc950eb9f51fa754'

    headers = {
        'Content-Type':'application/octet-stream',
        'Ocp-Apim-Subscription-Key':api_key
    }

    with open(image_path, 'rb') as image:
        image_data = image.read()

    response = requests.post(endpoint,
                             headers=headers,
                             data=image_data)
    
    response_json = response.json()
    print(response_json)

    return response_json

def change_image(image_path):
    if image_path is None:
        return None
    
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("NanumGothic.ttf", 30)
    except IOError:
        font = ImageFont.load_default()

    response_json = request_vision(image_path)
    # 받은 데이터를 파싱해서 image에 그린다.  
    if "objectsResult" in response_json and "values" in response_json["objectsResult"]:  
        for obj in response_json["objectsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            if "tags" in obj:  
                for tag in obj["tags"]:  
                    draw.text((x, y - 10), tag["name"], fill="red")
    
    if "tagsResult" in response_json and "values" in response_json["tagsResult"]:
        for obj in response_json["tagsResult"]["values"]:
            print(obj['name'], int(obj['confidence']*100))

    if "smartCropsResult" in response_json and "values" in response_json["smartCropsResult"]:  
        for obj in response_json["smartCropsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            draw.text((x, y - 10), str(obj['aspectRatio']), fill="red", font=font)  
                    
    return image

with gr.Blocks() as demo:
    input_image = gr.Image(label="이미지 선택", type="filepath", width=800)
    output_image = gr.Image(label="출력 이미지", type="pil", width=800, interactive=False)

    input_image.change(fn=change_image, inputs=[input_image], outputs=[output_image])

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


{'modelVersion': '2023-10-01', 'metadata': {'width': 816, 'height': 612}, 'tagsResult': {'values': [{'name': 'tool', 'confidence': 0.9927400946617126}, {'name': 'scissors', 'confidence': 0.9893608093261719}, {'name': 'office instrument', 'confidence': 0.863634467124939}, {'name': 'indoor', 'confidence': 0.701828122138977}, {'name': 'blue', 'confidence': 0.6598926782608032}]}, 'objectsResult': {'values': [{'boundingBox': {'x': 206, 'y': 123, 'w': 431, 'h': 351}, 'tags': [{'name': 'Scissors', 'confidence': 0.899}]}]}, 'smartCropsResult': {'values': [{'aspectRatio': 1.34, 'boundingBox': {'x': 34, 'y': 26, 'w': 748, 'h': 560}}]}}
206 123 431 351
tool 99
scissors 98
office instrument 86
indoor 70
blue 65
34 26 748 560
